In [1]:
import os
import numpy as np
from PIL import Image,ImageTk
from io import BytesIO
import requests
from tkinter import Tk,Label,Canvas,Entry,Button,NW
from keras.models import Sequential,load_model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image as image_utils
from tensorflow.keras.optimizers import Adam
import warnings
warnings.simplefilter("ignore")

In [2]:
model=Sequential()

model.add(Conv2D(32,(2,2),input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(2,2),input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(2,2),input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(2,2),input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512,(2,2),input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(1024,(2,2),input_shape=(224,224,3),activation="relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Dropout(rate=0.10))          
model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dense(6, activation="softmax"))
          

In [3]:
# model.summary()

In [4]:
# model.compile(optimizer='RMSprop',
#                 # loss='mse',  
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])
# model.compile(optimizer = Adam(lr = 0.001),
#                   loss = "categorical_crossentropy",
#                   metrics = ["accuracy"]) 

model.compile(optimizer = Adam(lr = 0.0001),
                  loss='mse',
                  metrics = ["accuracy"])
# model.compile(optimizer = Adam(
#                     learning_rate=0.001,
#                     beta_1=0.9,
#                     beta_2=0.999,
#                     epsilon=1e-07,
#                     amsgrad=False,
#                     name="Adam",),
#                     loss='mse',
#                     # loss='categorical_crossentropy',
#                     metrics = ["accuracy"])



In [5]:
train_datagen=ImageDataGenerator(rescale=1.0/255,
                                     # preprocessing_function = None,
                                     rotation_range = 60,
                                     # zoom_range = 0.1,
                                     # # horizontal_flip = True,
                                     # # vertical_flip = True,
                                     fill_mode = 'nearest'
                                     # shear_range = 0.1,
                                     # height_shift_range = 0.1
                                     # width_shift_range = 0.1                              
                                )
test_datagen=ImageDataGenerator(
                                rescale=1.0/255,
                                rotation_range = 60,
                                fill_mode = 'nearest'
                                )

In [6]:


training_dataset=train_datagen.flow_from_directory('thesis dataset_all/training_set',target_size=(224,224),batch_size=32,class_mode='categorical')
test_dataset=test_datagen.flow_from_directory('thesis dataset_all/test_set',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 213 images belonging to 6 classes.


In [7]:
model.fit(training_dataset,steps_per_epoch=152/32,epochs=2,validation_data=test_dataset,validation_steps=52/32)



Epoch 1/2
4/4 [==============================] - 10s 2s/step - loss: 0.1324 - accuracy: 0.3063 - val_loss: 0.1359 - val_accuracy: 0.3125
Epoch 2/2
4/4 [==============================] - 8s 2s/step - loss: 0.1270 - accuracy: 0.3904 - val_loss: 0.1358 - val_accuracy: 0.3281


In [8]:
model.save("model/model.h5")
model.save_weights("model/model_weights.h5")

In [9]:
image_width, image_height = 224, 224
model_path = "model/model.h5"
model_wight_path = "model/model_weights.h5"

final_model = load_model(model_path) 
final_model.load_weights(model_wight_path)

In [10]:
url=''
window=Tk()
window.title("Image Classifcation Using CNN")#
window.geometry("800x800")
label=Label(window,text="Please Enter your Custom URL",font=("Halvetica",16))
label.pack()

In [11]:
def Enter():
    global url
    label.configure()
    url=(User_input.get())
    print(url)
    
    response=requests.get(url)
    test_image=Image.open(BytesIO(response.content))
    put_image=test_image.resize((400,400))
    test_image=test_image.resize((224,224))
    
    img=ImageTk.PhotoImage(put_image)
    pic=Label(image=img)
    pic.pack()
    
    pic.image=img
    test_image=image_utils.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    
    results=model.predict_on_batch(test_image)
    
    if results[0][0]==1:
        res="dog ringworm"
    elif results[0][1]==1:
        res="ear mites"
    elif results[0][2]==1:
        res="fungal infection"
    elif results[0][3]==1:
        res="Leprosy in cat"
    elif results[0][4]==1:
        res="Leprosy in dog"
    elif results[0][5]==1:
        res="mange"
        
   
        
    output=Label(window,text=" Predicted Results: "+ res,font=("Halvetica",16))
    output.pack()

In [12]:
User_input=Entry()
User_input.pack()
button=Button(window,text=" Detect Image : " ,font=("Halvetica",16),command=Enter)
button.pack()
window.mainloop()